In [2]:
import pandas as pd
import numpy as np

store_info_data = pd.read_csv('Store-info.csv')
historical_data = pd.read_csv('Historical-transaction-data.csv')
testing_data = pd.read_csv('Testing-data.csv')

store_info_data.head()

,shop_id,shop_area_sq_ft,shop_profile
0,SHOP047,528,Moderate
1,SHOP009,676,High
2,SHOP083,676,Low
3,SHOP117,676,Low
4,SHOP042,676,Low


In [3]:
merged_df = pd.merge(historical_data, store_info_data, on='shop_id', how='left')

merged_df.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,shop_area_sq_ft,shop_profile
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2,678,Moderate
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2,668,Moderate
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2,678,Moderate
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2,678,Moderate
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5,668,Moderate


In [5]:
num_rows_with_null = merged_df.drop('shop_profile', axis=1).isnull().any(axis=1).sum()

print("Number of rows with null values (excluding shop_profile column):", num_rows_with_null)

Number of rows with null values (excluding shop_profile column): 41865


In [6]:
merged_df = merged_df.dropna(subset=['shop_profile'])

print(len(merged_df))

387341


In [7]:
null_count = merged_df['item_description'].isnull().sum()
print("Number of null values in item_description feature:", null_count)

Number of null values in item_description feature: 29614


In [9]:
both_notnull_count = (merged_df[['item_description', 'item_price']].isnull().all(axis=1)).sum()
print("Number of rows with non-null values for both item_description and item_price:", both_notnull_count)


Number of rows with non-null values for both item_description and item_price: 0


In [16]:
# create a dictionary mapping item prices to their most common item descriptions
price_to_mode = merged_df.groupby('item_description')['item_price'].apply(lambda x: x.mode()[0]).to_dict()
# Loop through each row in merged_df
for index, row in merged_df.iterrows():
    # Check if item_description is null
    if pd.isnull(row['item_description']):
        # Find the item name in price_to_mode that has the same price
        item_name = [key for key, value in price_to_mode.items() if value == row['item_price']]
        # If there is a matching item name, fill the null value in item_description
        if item_name:
            merged_df.at[index, 'item_description'] = item_name[0]

null_count = merged_df['item_description'].isnull().sum()
print("Number of null values in item_description feature:", null_count)

Number of null values in item_description feature: 12773


In [17]:
merged_df['item_price'] = merged_df.apply(lambda row: price_to_mode[row['item_description']] if pd.isnull(row['item_price']) else row['item_price'], axis=1)

null_count = merged_df['item_price'].isnull().sum()
print("Number of null values in item_price feature:", null_count)

Number of null values in item_price feature: 0


In [19]:
merged_df.dropna(subset=['item_description'], inplace=True)

null_count = merged_df['item_description'].isnull().sum()
print("Number of null values in item_description feature:", null_count)

Number of null values in item_description feature: 0


In [20]:
null_count = merged_df['transaction_date'].isnull().sum()
print("Number of null values in transaction_date feature:", null_count)

Number of null values in transaction_date feature: 0


In [21]:
null_count = merged_df['invoice_id'].isnull().sum()
print("Number of null values in invoice_id feature:", null_count)

Number of null values in invoice_id feature: 5178


In [25]:
null_count = merged_df['shop_area_sq_ft'].isnull().sum()
print("Number of null values in shop_area_sq_ft feature:", null_count)

Number of null values in shop_area_sq_ft feature: 0


In [26]:
merged_df = merged_df.drop("invoice_id",axis=1)

merged_df.head()

,item_description,transaction_date,customer_id,shop_id,item_price,quantity_sold,shop_area_sq_ft,shop_profile
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,BGXA,SHOP008,220,2,678,Moderate
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,IA25,SHOP112,220,2,668,Moderate
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,VN7V,SHOP008,160,2,678,Moderate
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,VN7V,SHOP008,150,2,678,Moderate
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,7S00,SHOP112,210,5,668,Moderate


In [28]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
merged_df['item_description'] = le.fit_transform(merged_df['item_description'])
merged_df['customer_id'] = le.fit_transform(merged_df['customer_id'])
merged_df['shop_profile'] = le.fit_transform(merged_df['shop_profile'])
merged_df.head()

,item_description,transaction_date,customer_id,shop_id,item_price,quantity_sold,shop_area_sq_ft,shop_profile
0,27,2021-12-11T00:00:00.000Z,50604,SHOP008,220,2,678,2
1,14,2021-10-17T00:00:00.000Z,80784,SHOP112,220,2,668,2
2,35,2021-12-13T00:00:00.000Z,139882,SHOP008,160,2,678,2
3,4,2021-12-13T00:00:00.000Z,139882,SHOP008,150,2,678,2
4,34,2021-10-23T00:00:00.000Z,34359,SHOP112,210,5,668,2


In [29]:
merged_df = merged_df.drop("shop_id",axis=1)

merged_df.head()

,item_description,transaction_date,customer_id,item_price,quantity_sold,shop_area_sq_ft,shop_profile
0,27,2021-12-11T00:00:00.000Z,50604,220,2,678,2
1,14,2021-10-17T00:00:00.000Z,80784,220,2,668,2
2,35,2021-12-13T00:00:00.000Z,139882,160,2,678,2
3,4,2021-12-13T00:00:00.000Z,139882,150,2,678,2
4,34,2021-10-23T00:00:00.000Z,34359,210,5,668,2


In [32]:
merged_df['transaction_month'] = pd.to_datetime(merged_df['transaction_date']).dt.month
merged_df = merged_df.drop("transaction_date",axis=1)

merged_df.head()

,item_description,customer_id,item_price,quantity_sold,shop_area_sq_ft,shop_profile,transaction_month
0,27,50604,220,2,678,2,12
1,14,80784,220,2,668,2,10
2,35,139882,160,2,678,2,12
3,4,139882,150,2,678,2,12
4,34,34359,210,5,668,2,10


In [37]:
# Select columns with numeric data types
numeric_cols = merged_df.select_dtypes(include=[int, float])

# Drop rows with negative values in any numeric column
merged_df = merged_df.drop(numeric_cols[(numeric_cols < 0).any(1)].index)


<ipython-input-37-e7cf8abd89dd>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  merged_df = merged_df.drop(numeric_cols[(numeric_cols < 0).any(1)].index)


In [40]:
from sklearn.feature_selection import SelectKBest, chi2

# Separate the features and target variable
X = merged_df.drop('shop_profile', axis=1)
y = merged_df['shop_profile']

# Use SelectKBest to select top 5 features based on chi-squared test
selector = SelectKBest(chi2, k=5)
X_new = selector.fit_transform(X, y)

# Get the scores and p-values for each feature
scores = selector.scores_
pvalues = selector.pvalues_

# Create a new DataFrame with the selected features and their scores/p-values
selected_features = X.columns[selector.get_support()]
feature_scores = pd.DataFrame({'Feature': selected_features, 'Score': scores[selector.get_support()], 'p-value': pvalues[selector.get_support()]})

# View the selected features and their scores/p-values
print(feature_scores)


            Feature         Score       p-value
0  item_description  2.162242e+02  1.115583e-47
1       customer_id  4.696994e+05  0.000000e+00
2        item_price  5.983814e+03  0.000000e+00
3     quantity_sold  1.397158e+02  4.582472e-31
4   shop_area_sq_ft  1.191486e+06  0.000000e+00


In [41]:
from sklearn.model_selection import train_test_split

# Splitting the dataframe into train and test sets
train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

# Displaying the number of rows in the train and test dataframes
print("Number of rows in train_df: ", train_df.shape[0])
print("Number of rows in test_df: ", test_df.shape[0])

Number of rows in train_df:  299647
Number of rows in test_df:  74912


In [42]:
# Define features and target variable
features = ['item_description', 'customer_id', 'item_price','quantity_sold','shop_area_sq_ft']
target = 'shop_profile'

# Create train_x and test_x dataframes with selected features
train_x = train_df[features]
test_x = test_df[features]

# Create train_y and test_y dataframes with target variable
train_y = train_df[target]
test_y = test_df[target]

In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# create KNN classifier object
knn = KNeighborsClassifier(n_neighbors=50)

# train the model on train_x and train_y data
knn.fit(train_x, train_y)

# make predictions on test_x data
y_pred = knn.predict(test_x)

# calculate accuracy score
accuracy = accuracy_score(test_y, y_pred)
f_score = f1_score(test_y, y_pred, average=None)

print("Accuracy:", accuracy*100)
print("F-score:", f_score * 100)

Accuracy: 49.61154421187527
F-score: [54.31719007 50.07742497 41.65454396]


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# create LogisticRegression classifier object
lr = LogisticRegression()

# train the model on train_x and train_y data
lr.fit(train_x, train_y)

# make predictions on test_x data
y_pred = lr.predict(test_x)

# calculate accuracy score
accuracy = accuracy_score(test_y, y_pred)

# calculate f1 score for each class
f_score = f1_score(test_y, y_pred, average=None)

print("Accuracy:", accuracy * 100)
print("F-score:", f_score * 100)


Accuracy: 39.79202263989748
F-score: [54.29620059 26.06119426  8.29850489]


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# create SVM classifier object with sigmoid kernel
svm = SVC(kernel='sigmoid')

# train the model on train_x and train_y data
svm.fit(train_x, train_y)

# make predictions on test_x data
y_pred = svm.predict(test_x)

# calculate accuracy score
accuracy = accuracy_score(test_y, y_pred)

# calculate f1 score for each class
f_score = f1_score(test_y, y_pred, average=None)

print("Accuracy:", accuracy * 100)
print("F-score:", f_score * 100)
